# HOMER DATA PROCESSING SCRIPT
The goal of this script is to take the output file of HOMER Pro Batch Runs (.xlsx file containing one optimal case/sheet) and produce a condensed data set + plots.

General flow:
1) Run HOMER study via batch process. Select "export results" and name the file.


2) Import file to this script.


3) Use Pandas to pull .xlsx data into a more usable format. 


4) Use collected data to build plots. 

Goals:


Have this script take a large HOMER study (>100 cases) and build one easily accessible database.


Potentially couple this script with a HOMER input reading script to do cartography w/HOMER geography + results.


In [36]:
# Block of dependencies and working directory
import matplotlib as plt
import conda
import numpy
import pandas as pd
import os 
#directory = 'd:/repo/homer/ClimateStudies/OutputProcessing/Data' #home PC
directory = 'c:/repo/homer/ClimateStudies/OutputProcessing/Data' #laptop

## Following this guide for xlsx reading
https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

In [3]:
#output = pd.read_excel('RevisedCase1.xlsx',sheet_name=None)#brings out all of the xlsx data as a dictionary
#output_keys = output.keys()  #grabs dictionary keys 
#type(output_keys)
#print(output_keys)
#data = output[output_keys]
#data["Cost/NPC\n($)"]

In [23]:
# reads, maps all sheets 
files = os.listdir(directory)
output_df = {} #creates empty dictionary
names = [] #creates empty list for future name storage
for file in files:
    if file.endswith('.xlsx'):
        xls = pd.ExcelFile(directory + '/' + file) #reads file
        names = names + xls.sheet_names
        for sheet_name in xls.sheet_names: #pulls individual sheets
            output_df[sheet_name] = xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
#display(output_df[sheet_name])

In [27]:
#pulling interesting data points from each 
opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)'])
n = len(names)
for i in range(n):
    name = names[i] # Grabs Sheet Name
    Location = name[5:-3]
    case_num = name[:5] # Grabs Case Number 
    KCC = name[-3:]
    KCC = KCC.lower()
    filtered_out = output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)']) #Filtering objects of interest
    filtered_out['CaseName'] = Location #adding actual case name to series
    filtered_out['CaseNum'] = case_num # adding case num to series
    filtered_out['KCC'] = KCC
    framed = filtered_out.iloc[[0]] #making the actual df
    framed.set_index(['CaseNum','CaseName'], inplace=True, drop=True) #building df w/ casename and num
    opt_out = pd.concat([framed,opt_out]) #concating all DFs together
opt_out.to_excel("LocationalVarianceTrimmed.xlsx")
display(opt_out)
#display()

Architecture/PV\n(kW)  Architecture/G1500  \
CaseNum CaseName                                               
Case4   WadenaMN            10727.029540                 7.0   
        VegasNV             19857.025383                 6.0   
        UPMich              17200.157069                 9.0   
        TexasOil            14815.152633                 6.0   
        Syracuse            33010.742566                 6.0   
...                                  ...                 ...   
Case2   BangorME             8203.981749                 NaN   
        AZYuma               8426.763084                 NaN   
        AtlantaGA            8398.530414                 NaN   
        Ashland              8068.539604                 NaN   
        Anchorage            7388.403933                 NaN   

                   Architecture/GenLarge\n(kW)  Architecture/1kWh LI  \
CaseNum CaseName                                                       
Case4   WadenaMN                        3000.0               25372.0   
        VegasNV                         2000.0               43517.0   
        UPMich                          3000.0               37649.0   
        TexasOil                        2000.0               35220.0   
        Syracuse                        3000.0               32103.0   
...                                        ...                   ...   
Case2   BangorME                        5000.0                1798.0   
        AZYuma                          5000.0                1414.0   
        AtlantaGA                       5000.0                1635.0   
        Ashland                         5000.0                1465.0   
        Anchorage                       5000.0                1094.0   

                   Cost/COE\n($)  KCC  
CaseNum CaseName                       
Case4   WadenaMN        0.138638  dfa  
        VegasNV         0.155043  bwh  
        UPMich          0.162512  dfb  
        TexasOil        0.131990  bsh  
        Syracuse        0.179424  dfb  
...                          ...  ...  
Case2   BangorME        0.070096  dfb  
        AZYuma          0.064478  bwh  
        AtlantaGA       0.068569  cfa  
        Ashland         0.069366  dfb  
        Anchorage       0.071984  dfc  

[186 rows x 6 columns]

In [39]:
opt_out.loc["Case4"]
plt.pyplot.scatter(opt_out.loc["Cost/COE\n($)"])

AttributeError: module 'matplotlib' has no attribute 'pyplot'